In [1]:
import pandas as pd
import numpy as np
from datetime import time
import pickle


# Task defined

1, A line

2, To wait

3, To charge

4, To do a crew pause


Rebalancing is considered a task when it comes to the vessel's itinerary, however, not a task when doing the coding.

# Dataframes

In [2]:
# Station and wharves dataframe
wharf_df = pd.read_csv('csv_inputs_original_copy/wharf_info.csv')

# lines dataframe
line_df = pd.read_csv('csv_inputs_original_copy/line_info.csv')
line_df['First_sailing'] = pd.to_datetime(line_df['First_sailing'], format='%H:%M')

# Wharf to wharf transit time dataframe
tt_df = pd.read_csv('csv_inputs_original_copy/rebalancing_times.csv',index_col='From/To')

# Headways dataframe
headway_df = pd.read_csv('csv_inputs_original_copy/headways.csv')

# vessels
vessel_df = pd.read_csv('csv_inputs_original_copy/vessel_info.csv')

# charging berths dataframe
charging_berth = pd.read_csv('csv_inputs_original_copy/charging_berths.csv')

# Other input required

In [3]:
## Other input required

# Simulation time parameters
initial_time = time(5,30)
period_length = 5 # min
total_operation_hours = 24 # hours

# nc, Minimum num of crew break
nc = 5 

# Dc, Crew break duration (fixed)
Dc = 60

# Tc, Maximum seperation time for crew breakings
Tc = 15*60

# rv+, charging rate
rv_plus = 2100*period_length/60/ 1100 # kW*h/kwh --> %

# rv defined later

# pc. Plugging/Unplugging time
pc = 2 # min

# Battery info, already organized into the csv file, here is just an explaination
# discharging rate/min / 1100kwh * time period = change of vessle between 1 time period
# rj_13 = 46/5/1100*period_length # percentage change per time period
# rj_20 = 167/10/1100*period_length
# rj_25 = 117/5/1100*period_length


# Sets

In [4]:
# Lset: Set of Lines
Lset = line_df['Line_No'].unique().tolist()

# Zset: Set of Sailing 
nl_ls= [len(headway_df[f'h{l}'].dropna().tolist())+1 for l in Lset]
s_ls = [list(range(1,nl+1)) for nl in nl_ls]
Zset = []

for line in Lset:
    for sailing in s_ls[line-1]:
        Zset.append(f'{line}_{sailing}')

# Vset: Set of Vessels
Vset = vessel_df['Vessel code'].unique().tolist()

# rv, discharging rate for revalancing, based on max speed of the vessel
rv = {}
for v in Vset:
    rv_value = vessel_df[vessel_df['Vessel code'] == v]['rv'].iloc[0]
    rv[v] = rv_value
    
# Wset: Set of Wharves
Wset = wharf_df['Wharf_No'].unique().tolist()

# Tset: Set of Time Periods
Tset = [i for i in range(1, total_operation_hours * 60 // period_length + 1)]

# Jset: Set of tasks
# B+, set of wharves to charge
Bplus = [wharf for wharf in charging_berth['Wharf_No'].unique().tolist()]
print(f'The set of wharves for vessels can charge, B+: {Bplus}')

# Non loading berths
original_non_loading_berths = wharf_df[wharf_df['Non_loading_berths'] != 0]['Wharf_No'].unique().tolist()

# Bc, set of wharves to crew pause, is copy of set B
Bc = ['cp_' + wharf for wharf in original_non_loading_berths] # or Bc = wharf_df[wharf_df['Crew_pause'] == 'Yes']['Wharf_No'].unique().tolist()    they result in the same set
print(f'The set of wharves for vessels can do crew pause, Bc: {Bc}')


# B, set of wharves to wait, any wharf with a charger belongs to B, and B contains wharves with original non loading berths
B = original_non_loading_berths.copy() 

for wharf in Bplus:
    if wharf in B: # wharf with a charger
        B.remove(wharf)
        B.append(f'phi_{wharf}') # mark as phi(w)
    else:
        B.append(f'phi_{wharf}')  # input directly

print(f'The set of wharves for vessels can wait, B: {B}')


# Jset = Lset + B + B+ + Bc
Jset = [ele for ele in Lset + B + Bc + Bplus]

# Dset: Set of possible first sailing time
# Defined later in the constraints part

The set of wharves for vessels can charge, B+: ['CQ2', 'CQ4', 'CQ5', 'Bar1', 'Bar2', 'Chs1', 'Cab1', 'SOP1', 'Pm1', 'WB1', 'RB1', 'CI1', 'PB1']
The set of wharves for vessels can do crew pause, Bc: ['cp_CQ1', 'cp_CQ2', 'cp_CQ3', 'cp_CQ4', 'cp_CQ5', 'cp_Bar1', 'cp_Bar2', 'cp_Bar4', 'cp_Bar5', 'cp_BSY1', 'cp_BSY2', 'cp_BSY3', 'cp_BSY4', 'cp_BSY5', 'cp_BSY6']
The set of wharves for vessels can wait, B: ['CQ1', 'CQ3', 'Bar4', 'Bar5', 'BSY1', 'BSY2', 'BSY3', 'BSY4', 'BSY5', 'BSY6', 'phi_CQ2', 'phi_CQ4', 'phi_CQ5', 'phi_Bar1', 'phi_Bar2', 'phi_Chs1', 'phi_Cab1', 'phi_SOP1', 'phi_Pm1', 'phi_WB1', 'phi_RB1', 'phi_CI1', 'phi_PB1']


In [5]:
Tset[-1]

288

# Functions

In [6]:
# S, set of station
S = wharf_df['Station'].unique().tolist()
print(f'The set of stations, S:{S}')

# PS
# Example: Circular quay
P_CQ = len(wharf_df[wharf_df['Station'] == 'Circular Quay']['Wharf_No'].unique())
print(f'Num of wharves in Circular Quay: P_CQ = {P_CQ}')

# Cw
# Example: Capacity of wharf 1 at Circular quay
C_CQ1 = wharf_df[(wharf_df['Station'] == 'Circular Quay')&(wharf_df['Wharf_No'] == 'CQ1')]['Loading_berths'].iloc()[0] + wharf_df[(wharf_df['Station'] == 'Circular Quay')&(wharf_df['Wharf_No'] == 'CQ1')]['Non_loading_berths'].iloc()[0]
print(f'Capacity of wharf 1 in Circular Quay: C_CQ1 = {C_CQ1}')

# S(w)
# Example: wharf CQ1's station
S_CQ1 = wharf_df[wharf_df['Wharf_No'] == 'CQ1']['Station'].iloc()[0]
print(f'The station corresponding to the wharf CQ1, S_CQ1 = {S_CQ1}')

The set of stations, S:['Circular Quay', 'Parramatta', 'Rydalmere', 'Sydney Olympic Park', 'Cabarita', 'Chiswick', 'Drummoyne', 'Cockatoo Island', 'Woolwich', 'Barangaroo', 'Pyrmont Bay', 'Mosman', 'Taronga Zoo', 'Watsons Bay', 'Rose Bay', 'Double Bay', 'Balmain Shipyard', 'Blackwattle Bay', 'Balmain East', 'McMahons Point', 'Milsons Point', 'Huntleys Point', 'Abbotsford', 'Kissing Point', 'Meadowbank', 'Kirribilli', 'North Sydney', 'Neutral Bay', 'Kurraba Point', 'Cremorne Point', 'South Mosman', 'Old Cremorne', 'Darling Point', 'Garden Island', 'Balmain', 'Birchgrove', 'Greenwich Point']
Num of wharves in Circular Quay: P_CQ = 5
Capacity of wharf 1 in Circular Quay: C_CQ1 = 2
The station corresponding to the wharf CQ1, S_CQ1 = Circular Quay


In [7]:
# Cw function

def cal_Cw(w):
    """
    Calculate the total capacity (number of berths) of a specific wharf.

    Parameters:
    w (str): The wharf identifier.

    Returns:
    int: The total capacity of the wharf, including loading and non-loading berths.

    Raises:
    ValueError: If the wharf identifier is not found or if required data is missing.
    """
    try:
        # Extract rows corresponding to the wharf
        wharf_data = wharf_df[wharf_df['Wharf_No'] == w]
        
        # Check if the wharf exists in the data
        if wharf_data.empty:
            raise ValueError(f"No data found for wharf {w}. Please check the wharf identifier.")
        
        # Calculate total capacity
        loading_berths = wharf_data['Loading_berths'].iloc[0]
        non_loading_berths = wharf_data['Non_loading_berths'].iloc[0]

        # Handle possible missing values for berths
        if pd.isna(loading_berths) or pd.isna(non_loading_berths):
            raise ValueError(f"Missing berth information for wharf {w}.")

        total_capacity = int(loading_berths) + int(non_loading_berths)
        return total_capacity

    except KeyError as e:
        raise KeyError(f"Missing required columns in the dataframe: {str(e)}")
    except Exception as e:
        raise Exception(f"An unexpected error occurred while calculating the capacity for wharf {w}: {str(e)}")



In [8]:
# Xi (S1, S2), travel time between S1 and S2 
# This parameter has no defined equation because in the following use the equation xi(j1,j1) can cover this parameter

# Example: Mosman to Taronga Zoo 
xi_Mos_TZ = tt_df.loc['Mosman', 'Taronga Zoo']
print(f'Travel time between Mosman and Taronga Zoo, the \u03be(Mos,TZ), is {xi_Mos_TZ} mins.')


Travel time between Mosman and Taronga Zoo, the ξ(Mos,TZ), is 5 mins.


In [9]:
# R(l) Function

def cal_Rl(l):
    """
    Calculate the route for a given line number from a DataFrame.
    
    Parameters:
    l (int): The line number for which the route is to be extracted.
    
    Returns:
    list: A list containing the sequence of stations for the line, excluding any NaN entries.
    
    Raises:
    ValueError: If the line number is not found in the DataFrame or input is not an integer.
    """
    
    # Check input type to be an integer
    if not isinstance(l, int):
        raise ValueError("Line number must be an integer.")
    
    # Check if line number exists
    if l not in line_df['Line_No'].values:
        raise ValueError(f"Line number {l} is not found in the DataFrame.")
    
    try:
        route_data = line_df[line_df['Line_No'] == l][['O', 'I', 'T']].iloc[0]
        # Filter out NaN values and convert to list
        R_l = [station for station in route_data if not pd.isna(station)]
        return R_l
    except IndexError:
        raise ValueError(f"No data available for line number {l}.")
    except KeyError:
        raise ValueError("DataFrame must include 'Line_No', 'O', 'I', 'T' columns.")


# Example: line1
print(f'The stations visited by line 1, R_l1, is {cal_Rl(1)}')


The stations visited by line 1, R_l1, is ['Circular Quay', 'Mosman']


In [10]:
# C(l,S) Function

def cal_C_lS(S):
    """
    Calculate the set of usable wharves for a given station of a line.
    
    Parameters:
    S (str): The station name for which the set of wharves is to be calculated.
    
    Returns:
    list: A list of unique wharf numbers that can be used at the given station.
    
    Raises:
    ValueError: If the station does not exist in the DataFrame or the input is not a string.
    """
    # Check if input is a string
    if not isinstance(S, str):
        raise ValueError("Station name must be a string.")
    
    # Check if station exists in DataFrame
    if S not in wharf_df['Station'].values:
        raise ValueError(f"Station {S} is not found in the DataFrame.")
    
    try:
        # Extract unique wharves for the station
        C_lS = wharf_df[wharf_df['Station'] == S]['Wharf_No'].unique().tolist()
        return C_lS
    except KeyError:
        raise ValueError("DataFrame must include 'Station' and 'Wharf_No' columns.")
    

# Example: Circular Quay
print('The wharves in Circular Quay that can be used are:', cal_C_lS('Circular Quay'))

The wharves in Circular Quay that can be used are: ['CQ1', 'CQ2', 'CQ3', 'CQ4', 'CQ5']


In [11]:
# dw(l,S), dwelling time
# Example: dwelling time for line1 at terminus, Mosman
dw_l1_Mos = int(line_df[line_df['Line_No'] == 1]['dw_T'].iloc()[0]//period_length+1)
print(f'The dwelling time for line 1 at Mosman, dw(l1,Mos),is {dw_l1_Mos} time periods.')

The dwelling time for line 1 at Mosman, dw(l1,Mos),is 2 time periods.


In [12]:
# Hl, headway
# Example: line 1
H_l1 = headway_df['h1'].dropna().tolist()
print(f'The headways of sailings for line1, H_l1, is {H_l1} in mins.')

The headways of sailings for line1, H_l1, is [30.0, 30.0, 30.0] in mins.


In [13]:
# Sv,starting station of vessel v # QUESTIONS HERE: Will this be given to us ?????????????????????????????????
# Example: Vessel FF1.
S_ff1 = vessel_df[vessel_df['Vessel code'] == 'FF1']['Sv'].iloc()[0]

# li(v)
# Example: Vessel FF1.
vessel_ff1 = vessel_df[vessel_df['Vessel code'] == 'FF1']
routes_served = [route for route in vessel_df.columns[2:-1] if vessel_ff1.iloc[0][route] == 'Yes']
# print("Routes served by FF1:", routes_served)

li_ff1 = line_df[(line_df['Route_No'].isin(routes_served))&(line_df['O'] == S_ff1)]['Line_No'].tolist()
print(f"Line served by FF1 staring from {S_ff1}, li_ff1 : {li_ff1}")


Line served by FF1 staring from Circular Quay, li_ff1 : [1, 3, 7, 9, 10, 12, 14]


In [14]:
# Sv(v) Function

def cal_Sv(v):
    """
    Retrieve the starting station for a given vessel identified by its vessel code.
    
    Parameters:
    v (str): The vessel code for which the starting station is to be retrieved.
    
    Returns:
    str: The starting station of the vessel.
    
    Raises:
    ValueError: If the vessel code does not exist in the DataFrame or the input type is incorrect.
    """
    # Validate input type
    if not isinstance(v, str):
        raise ValueError("Vessel code must be a string.")
    
    # Check if the vessel code exists in the DataFrame
    if v not in vessel_df['Vessel code'].values:
        raise ValueError(f"Vessel code {v} is not found in the DataFrame.")
    
    try:
        # Extract the starting station for the vessel
        S_v = vessel_df[vessel_df['Vessel code'] == v]['Sv'].iloc[0]
        return S_v
    except KeyError:
        raise ValueError("DataFrame must include 'Vessel code' and 'Sv' columns.")
    except IndexError:
        raise ValueError(f"No data available for vessel code {v}. The vessel might not be listed.")

# Example: Vessel FF1.
print(f"Vessel FF1 staring from {cal_Sv('FF1')}")

Vessel FF1 staring from Circular Quay


In [15]:
def cal_li(v):
    """
    Calculate the lines that a given vessel can serve, based on its starting station and the routes it can serve.
    
    Parameters:
    v (str): The vessel code for which lines are to be calculated.
    
    Returns:
    list: A list of line numbers that the vessel can serve starting from its designated station.
    
    Raises:
    ValueError: If the vessel code does not exist in the DataFrame or if the DataFrame structure is incorrect.
    """
    # Validate input type
    if not isinstance(v, str):
        raise ValueError("Vessel code must be a string.")
    
    # Check if vessel code exists in the DataFrame
    if v not in vessel_df['Vessel code'].values:
        raise ValueError(f"Vessel code {v} is not found in the DataFrame.")
    
    try:
        # Extract rows for the vessel and find routes served
        vessel_row = vessel_df[vessel_df['Vessel code'] == v].iloc[0]
        routes_served = [route for route in vessel_df.columns[2:-1] if vessel_row[route] == 'Yes']
        li_v = line_df[line_df['Route_No'].isin(routes_served)]['Line_No'].tolist() # DO NOT DEPEND ON THE Sv

        
        # # Does the li depend on Sv? I have written two versions here, and currently, I am using the version (code above) where it does not depend on Sv.??????????????????????????????????????????
        # # Retrieve the starting station for the vessel
        # S_v = cal_Sv(v)
        # # Find lines that start from S_v and are in routes_served
        # li_v = line_df[(line_df['Route_No'].isin(routes_served)) & (line_df['O'] == S_v)]['Line_No'].tolist()

        return li_v
    except KeyError:
        raise ValueError("DataFrame must include 'Vessel code', 'Route_No', 'O', and necessary route columns.")
    except IndexError:
        raise ValueError(f"No data available for vessel code {v}.")
    
# Example: Vessel FF1.
print(f"Line served by FF1, li_ff1 : {cal_li('FF1')}")


Line served by FF1, li_ff1 : [1, 2, 3, 4, 7, 8, 9, 10, 11, 12, 13, 14, 15]


In [16]:
# Rv
line_df['rj']

0     0.041818
1     0.041818
2     0.041818
3     0.041818
4     0.000000
5     0.000000
6     0.041818
7     0.041818
8     0.041818
9     0.041818
10    0.041818
11    0.041818
12    0.041818
13    0.041818
14    0.041818
15    0.106364
16    0.106364
17    0.041818
18    0.041818
Name: rj, dtype: float64

In [17]:
# D(l) Function

def cal_D(l):
    '''
    Determines the allowable time periods for the first sailing of a specified line, 
    relative to the given initial simulation time and allowed_latitude.

    Parameters:
    l (int): Line number.

    Returns:
    list: A list of allowable time periods, D(l), during which the specified line's
          first sailing is considered permissible.

    Example:
    For line l1 and an initial simulation time of 5:00 AM, D(l1) might include time periods
    where the first sailing time of line l1 falls within a certain allowable range.
    
    '''
    first_sailing_time = line_df[line_df['Line_No'] == l]['First_sailing'].iloc()[0]
    delta_minutes = (first_sailing_time.hour * 60 + first_sailing_time.minute) - (initial_time.hour * 60 + initial_time.minute)

    allowed_latitude = 15 # min
    # Create a set of allowable time period numbers
    D_l = list(range((delta_minutes - allowed_latitude) // period_length + 1, ((delta_minutes + allowed_latitude) // period_length + 1) + 1))
    # print(f"The set of time for line {l}'s first sailing, D(l1): {D_l}")
    return D_l

# Example: line 1
print(f"The set of time for line 1's first sailing, D(l1): {cal_D(1)}")


The set of time for line 1's first sailing, D(l1): [13, 14, 15, 16, 17, 18, 19]


In [18]:
# h(s,d) Function

def cal_h(s, d, line):
    """
    Calculate the s-th sailing time starting from time 'd' for a specified 'line'.

    This function retrieves headway periods for a specific line from a dataframe,
    constructs a list of sailing times starting from time 'd', and returns the s-th
    sailing time based on these intervals. It is used to project sailing schedules
    based on a specified headway and start day.

    Parameters:
    s (int): The order of the sailing time to be retrieved (1st, 2nd, etc.).
    d (int): The day from which sailing starts.
    line (int): The line number for which headway data is to be used.

    Returns:
    int or None: The s-th sailing time in terms of the number of time periods
                 since day 'd', or None if the index 's' is out of range or
                 an error occurs in processing.

    Note:
    The function assumes `period_length` as a global variable that denotes the
    length of each time period within the operational schedule.
    """

    try:
        # Retrieve the headway periods for the specified line and drop missing values
        h = headway_df[f'h{line}'].dropna().tolist()
        h_sd_ls = [d]  # Start the list with the initial day 'd'

        # Calculate subsequent sailing times based on headway periods
        for sailing_headway in h:
            num_time_period = int(sailing_headway // period_length + 1)  # round up
            h_sd_ls.append(h_sd_ls[-1] + num_time_period)

        # Return the s-th sailing time if within bounds
        if s-1 < len(h_sd_ls):
            return h_sd_ls[s-1]
        else:
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


# Example: 2nd sailing of line 1
print(f'The second sailing, h(2,1), starts at the {cal_h(2,22,1)}th time preiod.')

The second sailing, h(2,1), starts at the 29th time preiod.


In [19]:
# mu(j) Function

def cal_mu(j):
    """
    Calculate the duration in time periods for a given task j.
    
    Parameters:
    j (int or str): The task identifier which can be a line number, crew pause, or waiting task.
    
    Returns:
    int or None: The number of time periods the task takes, or None if the task is unrecognized.
    
    Raises:
    ValueError: If the input or required global variables are improperly configured.
    """
    try:
        if not isinstance(j, (int, str)):
            raise ValueError("Task identifier must be an integer or string.")
        
        # Check if j is in lists
        if j in Lset:
            mu_j = line_df[line_df['Line_No'] == j]['Line_duration'].iloc()[0] // period_length + 1
        elif j in Bc:
            mu_j = Dc // period_length + 1
        elif j in Bplus or j in B:
            mu_j = 1
        else:
            # If task j is unrecognized
            return None
        return mu_j
    except KeyError as e:
        raise ValueError(f"Missing data for task {j}: {str(e)}")
    except Exception as e:
        raise Exception(f"An unexpected error occurred: {str(e)}")
    
# Example1: crew puse
print('The minimum number of period for crew puse, \u03BC(cp), is ',cal_mu('cp_CQ1'))

# Example2: line 1
print(f'The minimum number of period for line 1, \u03BC(l1), is {cal_mu(1)}.')

The minimum number of period for crew puse, μ(cp), is  13
The minimum number of period for line 1, μ(l1), is 5.


In [20]:
# q_vjt Function

def cal_q(v, j, t):
    """
    Calculate the battery change rate qv,j,t for a vessel v performing task j at time t.

    Parameters:
    v (str): Identifier for the vessel.
    j (str or int): Identifier for the task.
    t (int): Time unit after the start of the task.

    Returns:
    float: The rate of battery change at time t.
    """

    if j in Bplus: # the task is charging, return the charging rate
        return rv_plus
    elif j in B: # the task is the first or last period of charging
        epsilon = 1-pc/period_length # REQUIRE CHECK: Do I understand this correctly ????????????????????????????????
        return epsilon * rv_plus
    elif j in B: # the task is waiting
        return 0
    elif j in Lset: # the task is a line
        l = j
        line_data = line_df[line_df['Line_No'] == l]
        R_l = cal_Rl(l)
        stops = R_l[1:] # remove the origin station
        # Check if it's during a stop
        if len(stops) == 1: # No intermediate stops
            a = int(line_data['Time_underway_to_T'].iloc[0] // period_length + 1)
            dw = int(line_data['dw_T'].iloc[0] // period_length + 1)
            if t in list(range(a,(a+dw)+1)): # at stop
                return 0 
            else:
                rj = line_data['rj'].iloc()[0]
        elif len(stops) == 2: # with intermediate stops
            a1 = int(line_data['Time_underway_to_I'].iloc[0] // period_length + 1)
            dw1 = int(line_data['dw_I'].iloc[0] // period_length + 1)
            a2 = int(line_data['Time_underway_to_T'].iloc[0] // period_length + 1)
            dw2 = int(line_data['dw_T'].iloc[0] // period_length + 1)

            if t in list(range(a1,(a1+dw1)+1)) + list(range(a2,(a2+dw2)+1)):
                return 0
            
            else:
                rj = line_data['rj'].iloc()[0]
        return -rj  # Negative because it's a consumption rate
    else: # the vessel is rebalncing, rv will be captured by the constraint
        return 0


In [21]:
## New function help with the xi(j1,j2) and xi0(v,j) functions

def get_task_location(j, type):
    """
    Retrieve the starting station or wharf location for a given task.

    Parameters:
    j (str or int): Task identifier, which could be a line number or a specific wharf/task identifier.
    type (int): Type of location required (0 for start or -1 for end).

    Returns:
    str: The starting station or wharf associated with the task.

    Raises:
    ValueError: If the task identifier is unrecognized, necessary data is missing, or parameters are invalid.
    IndexError: If expected data is not available in the DataFrame.
    """
    try:
        if type not in [0, -1]:
            raise ValueError("Type parameter must be int, 0 for start or -1 for end.")

        if j in Lset:
            task_stations = cal_Rl(j)
            if not task_stations:
                raise ValueError(f"No stations found for line {j}.")
            task_station = task_stations[type]
        elif j in Bc or j in B or j in Bplus:
            task_wharf = j.split('_')[-1]
            task_station_df = wharf_df[wharf_df['Wharf_No'] == task_wharf]
            if task_station_df.empty:
                raise ValueError(f"No station found for wharf identifier {task_wharf} from task {j}.")
            task_station = task_station_df['Station'].iloc[0]
        else:
            raise ValueError(f"Task identifier {j} is unrecognized or does not belong to known task sets.")
        
        return task_station
    except IndexError as e:
        raise IndexError(f"Data retrieval error for task {j}: {str(e)}")
    except Exception as e:
        raise Exception(f"An error occurred while retrieving the location for task {j}: {str(e)}")


In [22]:
# xi(j1,j2) function 
# required time for j1_end and j2_start

def cal_xi(j1, j2):
    """
    Calculate the rebalancing time needed to travel from the end of task j1 to the start of task j2.

    Parameters:
    j1 (str or int): Task identifier for the first task.
    j2 (str or int): Task identifier for the second task.

    Returns:
    int: The number of time units required to transition from the end of j1 to the start of j2.

    Raises:
    ValueError: If any of the task identifiers are unrecognized, if location data is missing,
                or if there is no travel time data available between the two tasks.
    """
    try:
        # Retrieve the ending location of j1 and starting location of j2
        end_location_j1 = get_task_location(j1, -1)
        start_location_j2 = get_task_location(j2, 0)

        # Check if any location data is missing
        if not end_location_j1 in tt_df.columns or not start_location_j2 in tt_df.columns:
            travel_time = 24*60+1 # Very long time --> ensure no rebalancing happend between the two station

        # Fetch travel time from the DataFrame based on the locations
        else:
            travel_time = tt_df.loc[end_location_j1, start_location_j2]

        # Calculate time units required for transition
        xi_j1_j2 = travel_time // period_length + 1
        return xi_j1_j2
    except Exception as e:
        raise Exception(f"An error occurred while calculating transition time from {j1} to {j2}: {str(e)}")

# Example: j1: line1 will end at Mosman, j2: 'cp_CQ1', crew pause at CQ1, ci(j1,j2) will be 15 mins, thus 15//5 + 1 = 4 time periods 

cal_xi(1,'cp_CQ1')

4

In [23]:
# xi0(v,j2) function 
def cal_xi0(v, j):
    """
    Calculate the time periods required for vessel v to travel from its starting position to the starting point of task j.
    If the starting position is the same as the task location, xi0 is zero.
    
    Parameters:
    v (str): The vessel identifier.
    j (str or int): The task identifier, which could be a line or a specific wharf.
    
    Returns:
    int: The time period required to travel from the vessel's starting wharf to the wharf where task j begins, or zero if they are the same.
    
    Raises:
    ValueError: If input data is missing or incorrect, or if travel times are not found in the DataFrame.
    """
    try:
        # Fetch the starting station for vessel v
        S_v = cal_Sv(v)
        # Fetch the starting station of the task using the refined function
        task_station = get_task_location(j,0)

        # If the starting point and task location are the same, return zero
        if S_v == task_station:
            return 1
        
        elif not S_v in tt_df.columns or not task_station in tt_df.columns:
            travel_time = 24*60+1 # Very long time --> ensure no rebalancing happend between the two station 

        else: # Fetch travel time from the travel time DataFrame
            travel_time = tt_df.loc[S_v, task_station]

        # Calculate periods
        xi0 = travel_time // period_length + 1
        return int(xi0)
    except KeyError as e:
        raise KeyError(f"DataFrame column missing: {str(e)}")
    except Exception as e:
        raise Exception(f"An error occurred: {str(e)}")

# Example: vessel FF1 do line 1, circular quaty --> circular quay, thus xi0 should be 0
cal_xi0('FF1', 1)


1

In [24]:
def cal_C(j):
    """
    Calculate all wharves that could be used given task j.
    If j is a line, aggregates wharves from all stations visited by the line.
    If j is a wharf (either in B or Bc), returns just that wharf.

    Parameters:
    j (int): The task identifier, which could be a line number or a wharf.

    Returns:
    list: A list of all wharves usable for the task, or None if task is unrecognized.

    Raises:
    Exception: Raises an exception with a descriptive message if any error occurs.
    """
    try:
        C_j = []
        if isinstance(j, int) and j in Lset:
            R_l = cal_Rl(j)  # stations visited by the line
            for S in R_l:
                C_lS = cal_C_lS(S)
                C_j.extend(C_lS)  # Use extend to avoid nested lists
        elif isinstance(j, str) and (j in Bc or B or Bplus):
            C_j.append(j.split('_')[-1])
        else:
            raise ValueError(f"Task {j} is unrecognized or inappropriate data type")

        return C_j
    except Exception as e:
        raise Exception(f"An error occurred: {str(e)}")


# Example: for line 1
print(f'the wharves for line1 that can be used are: {cal_C(1)}')

# Example: for crew pause
print(f'the wharves for crew pause at CQ1 that can be used are:',cal_C('cp_CQ1') )

the wharves for line1 that can be used are: ['CQ1', 'CQ2', 'CQ3', 'CQ4', 'CQ5', 'Mos1']
the wharves for crew pause at CQ1 that can be used are: ['CQ1']


In [25]:
# delta(j,w)
# Set of time t when w will be occupied because of j

# if j is a line
# Example: line 1 (no intermediate stop)
a = int(line_df[line_df['Line_No'] == 1]['Time_underway_to_T'].iloc()[0]//period_length + 1)
dw_T = int(line_df[line_df['Line_No'] == 1]['dw_T'].iloc()[0]//period_length + 1)


delta_l1_w = list(range(a-1,(a+dw_T-1+1)+1)) # assume safety buffer 1 time period, and at least including range(a,(a+dw_T-1)+1)
print(f'For line1, the time terminus wharf has been occupied is {delta_l1_w}')

# Example: line 7 (with intermediate stop)
a1 = int(line_df[line_df['Line_No'] == 7]['Time_underway_to_I'].iloc()[0]//period_length + 1)
a2 = int(line_df[line_df['Line_No'] == 7]['Time_underway_to_T'].iloc()[0]//period_length + 1)
dw_I = int(line_df[line_df['Line_No'] == 7]['dw_I'].iloc()[0]//period_length + 1)
dw_T = int(line_df[line_df['Line_No'] == 7]['dw_T'].iloc()[0]//period_length + 1)

delta_l7_iw = list(range(a1-1,(a1+dw_I-1+1)+1)) # assume safety buffer 1 time period, and at least including range(a1,(a1+dw_I-1)+1)
delta_l7_tw = list(range(a2-1,(a2+dw_T-1+1)+1) )# assume safety buffer 1 time period, and at least including range(a2,(a2+dw_T-1)+1)
print(f'For line7, the time intermediate stop wharf has been occupied is {delta_l7_iw}, the time terminus wharf has been occupied is {delta_l7_tw}')

# if j is not a line
# Example: crew pause
delta_j_j = [i for i in range(cal_mu('cp_CQ1'))]
print(f'For crew pause, the time a wharf has been occupied is {delta_j_j}') 


For line1, the time terminus wharf has been occupied is [3, 4, 5, 6]
For line7, the time intermediate stop wharf has been occupied is [4, 5, 6, 7], the time terminus wharf has been occupied is [5, 6, 7, 8]
For crew pause, the time a wharf has been occupied is [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]


In [26]:
# delta(j,w) Function

def cal_delta(j, w):
    """
    Calculate the set of times the wharf w will be occupied due to task j starting at time t0.
    For lines, excludes the last station from consideration and includes a safety buffer.
    For specific tasks like B, Bplus, or Bc, directly uses the task's duration.

    Parameters:
    j (int or str): The task identifier, which could be a line number or a specific wharf/task identifier.
    w (str): The wharf identifier.

    Returns:
    list: A list of tuples (wharf, time) indicating times the wharf is occupied.

    Raises:
    ValueError: If the task identifier or wharf is unrecognized, or if the wharf does not belong to the task.
    KeyError: If necessary data columns are missing from the data frames.
    IndexError: If data extraction based on indices fails.
    """
    
    try:
        if isinstance(j, int) and j in Lset:
            l = j # task is a line
            line_data = line_df[line_df['Line_No'] == l]
            safety_buffer = 1  # Assume a safety buffer of 1 time period
            R_l = cal_Rl(j)  # Stations visited by the line

            # Exclude the first and last station # QUESTION HERE: Why do we want to remove the last station here???????????????????????????????
            stations = R_l[1:-1]

            # Iterate through intermediate stations only
            for station in stations: # only one element here
                wharves = cal_C_lS(station)
                if w in wharves:
                    a = int(line_data['Time_underway_to_I'].iloc[0] // period_length + 1)
                    dw = int(line_data['dw_I'].iloc[0] // period_length + 1)
                    delta_j_w = [(w, time) for time in range(a - safety_buffer, (a + dw - 1 + safety_buffer) + 1)]
                    return delta_j_w

            # raise ValueError(f"Wharf {w} is not available for the task {j} at any intermediate stops.")
            return []

        elif isinstance(j, str) and (j in Bc or j in B or j in Bplus):
            if w != j.split('_')[-1]:
                raise ValueError(f"Task {j} should occupy its own wharf {j.split('_')[-1]}, not {w}.")
            mu_j = cal_mu(j)
            delta_j_w = [(w, time) for time in range(mu_j)]
            return delta_j_w

        else:
            raise ValueError(f"Task {j} is unrecognized or has an inappropriate data type.")

    except KeyError as e:
        raise KeyError(f"Missing data column: {str(e)}")
    except IndexError as e:
        raise IndexError(f"Data extraction error: {str(e)}")
    except Exception as e:
        raise Exception(f"Unexpected error occurred: {str(e)}")

# Example: line 7 using Bar1 
cal_delta(7,'Bar1')

[('Bar1', 4), ('Bar1', 5), ('Bar1', 6), ('Bar1', 7)]

In [27]:
# Error Output Example

# cal_delta('cp_CQ1','CQ2')

In [28]:
# A(l)
# Given a line l, we denote by A(l) its last station.
# Example: line1's A(l)
cal_Rl(1)[-1]

'Mosman'

In [29]:
# F (l) Function

def cal_F(l):
    """
    Calculate the number of time periods from the start of a sailing until arrival at the last station of line l.
    
    Parameters:
    l (int): Line
    
    Returns:
    int: Time periods until arrival at the last station
    
    Raises:
    ValueError: If line number does not exist or required data is missing.
    """
    try:
        if not isinstance(l, int):
            raise ValueError("Line number must be an integer.")
        
        time_underway_to_T = line_df[line_df['Line_No'] == l]['Time_underway_to_T'].iloc()[0]
        F_l = time_underway_to_T // period_length + 1
        return F_l
    except IndexError:
        raise ValueError(f"No data available for line number {l}.")
    except KeyError:
        raise ValueError("Missing 'Time_underway_to_T' in line_df.")

# muF(l)Function

def cal_muF(l):
    """
    Calculate the number of time periods a wharf is occupied at the last station by line l, including any safety buffer.
    
    Parameters:
    l (int): Line number
    
    Returns:
    int: Time periods a wharf is occupied
    
    Raises:
    ValueError: If line number does not exist or required data is missing.
    """
    try:
        if not isinstance(l, int):
            raise ValueError("Line number must be an integer.")
        
        dw_T = line_df[line_df['Line_No'] == l]['dw_T'].iloc()[0]
        muF_l = dw_T // period_length + 1
        return muF_l
    except IndexError:
        raise ValueError(f"No data available for line number {l}.")
    except KeyError:
        raise ValueError("Missing 'dw_T' in line_df.")

# Example: line1:
print(f'The number of time periods from the start of a sailing until arrival at the last station of line 1, F(1), is {cal_F(1)}')
print(f'The number of time periods a wharf is occupied at the last station by line 1, \u03BC_F(1), is {cal_muF(1)}')

The number of time periods from the start of a sailing until arrival at the last station of line 1, F(1), is 4
The number of time periods a wharf is occupied at the last station by line 1, μ_F(1), is 2


In [30]:
# phi(j,t) Function

def cal_phi(j, t):
    """
    Calculate a set of time periods within which if a task j starts, it will still be ongoing at time t.
    
    Parameters:
    j (int or str): The task identifier.
    t (int): The time period at which task j is still ongoing if started within the returned set.
    
    Returns:
    list: A list of time periods representing possible start times for task j to be ongoing at time t.
    
    Raises:
    ValueError: If the inputs are not valid or the time period is out of expected range.
    """
    try:
        # Check if t is a valid input
        if not isinstance(t, int) or t < 1:
            raise ValueError("Time period t must be a positive integer.")

        mu_j = cal_mu(j)  # Duration that task j occupies
        if mu_j is None:
            raise ValueError(f"No duration found for task {j}. It may be unrecognized.")

        # Compute the range of start times
        phi_j_t = list(range(max(1, t - mu_j + 1), t + 1))
        return phi_j_t
    except Exception as e:
        raise ValueError(f"An error occurred calculating φ(j, t): {str(e)}")

# Example: doing crew pause 
t = 100 # assumed 
print('The set of time periods ',cal_phi('cp_CQ1', t),' for crew pause to be still ongoing at the 100th time period')

The set of time periods  [88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]  for crew pause to be still ongoing at the 100th time period


In [31]:
# f_j(j) Function

def cal_f(j):
    """
    Determines the latest feasible start time for task `j` so that it finishes before the day ends.

    Parameters:
    j (str or int): Task identifier. Can be an integer for line tasks or a string for specific wharves or tasks.

    Returns:
    int or None: The last valid starting period for the task, or None if the task cannot be completed in a day or an error occurs in processing.
    """
    try:
        # Validate that Tset is properly defined and not empty
        if not Tset:
            raise ValueError("Tset is not defined or is empty.")
        
        last_period = Tset[-1]  # Last time period in the set
        
        # Calculate mu(j) based on the type of task
        if j in Lset:
            mu_j = line_df[line_df['Line_No'] == j]['Line_duration'].iloc()[0] // period_length + 1
        elif j in Bc:
            mu_j = Dc // period_length + 1
        elif j in Bplus or j in B:
            mu_j = 1  # Fixed duration for waiting tasks
        else:
            # If task j is unrecognized
            return None

        # Calculate the latest feasible start time
        last_start_time = last_period + 1 - mu_j
        return last_start_time
    except Exception as e:
        print(f"An error occurred: {e}")
        return None


print('The last time for crew pause to be schedule is at the ',cal_f('cp_CQ1')  ,'th time peried starts')  

The last time for crew pause to be schedule is at the  276 th time peried starts


In [32]:
# G(l)   Function

def cal_G(j):
    """
    Calculate the set of valid start times for task j.
    
    If j is a line, G(j) includes times based on headways and initial days from D(l).
    If j is a crew pause, waiting, or charging, G(j) includes all times in Tset.

    Parameters:
    j (int or str): The task identifier.

    Returns:
    list: List of valid start times for the task j.

    Raises:
    ValueError: If task j is unrecognized or essential data is missing.
    """
    
    try:
        G_j = []
        if j in Lset:  # a line
            headways = headway_df.get(f'h{j}', pd.Series()).dropna().tolist()
            if not headways:
                raise ValueError(f"No headway data available for line {j}")
            D_l = cal_D(j) 
            for d in D_l:
                G_j.append(d)
                current_time = d
                for h in headways:
                    num_time_period = int(h // period_length + 1)
                    current_time += num_time_period
                    G_j.append(current_time)
        elif j in Bc or j in B or j in Bplus:  # crew pause / waiting / charging
            G_j = Tset.copy() 
        else:
            raise ValueError(f"Task {j} is unrecognized or not handled.")

        return G_j
    except Exception as e:
        raise ValueError(f"An error occurred processing task {j}: {str(e)}")

# Example:


print(f'G(l1), the set of valid times for line 1 can be started is {cal_G(1)}')


G(l1), the set of valid times for line 1 can be started is [13, 20, 27, 34, 14, 21, 28, 35, 15, 22, 29, 36, 16, 23, 30, 37, 17, 24, 31, 38, 18, 25, 32, 39, 19, 26, 33, 40]


In [33]:
# H(v,j) Function

def cal_H(v, j):
    """
    Calculate the set of feasible start times H(v,j) for vessel v to start task j.
    
    Parameters:
    v (int): Index of the vessel.
    j (int or str): Index of the task, a line or a wharf.
    
    Returns:
    list: List of feasible times or None if start point is not determined.
    """
    try:
        S_v = cal_Sv(v)
        if pd.isna(S_v):
            print(f'The start point of the vessel {v} has not been determined. Please determine it first.')
            return None

        xi0_vj = cal_xi0(v, j)
        f_j = cal_f(j)
        G_j = cal_G(j)

        # Filter G(j) to find all t such that xi0(v, j) ≤ t ≤ f(j)
        H_vj = [t for t in G_j if xi0_vj <= t <= f_j]

        return H_vj
    except Exception as e:
        print(f"An error occurred while calculating H(v, j): {str(e)}")
        return None

# Example: 
print(f'The set of times vessel FF1 can start doing crew pause at CQ1, is',cal_H('FF1', 'cp_CQ1'))

The set of times vessel FF1 can start doing crew pause at CQ1, is [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 2

In [34]:
# F(j,t)
# set of tasks that can be performed-and-finished after task j

def cal_taskF(j, t):
    """
    Calculate the set of tasks that can be performed and finished after task j if j started at time t.

    Parameters:
    j (str or int): The task identifier for which subsequent tasks are calculated.
    t (int): The start time of task j.

    Returns:
    list: A list of tasks that can be started and completed after task j.

    Raises:
    ValueError: If the start time 't' is not valid or if 'j' does not have a feasible completion time.
    KeyError: If there are any missing required data fields or calculations.
    """
    try:
        # Validate input
        if not isinstance(t, int) or t < 0:
            raise ValueError("Start time 't' must be a non-negative integer.")
        
        feasible_tasks = []
        f_j = cal_f(j)  # Calculate the latest feasible start time for task j

        if t > f_j:
            return feasible_tasks  # Return an empty list if j can't be completed

        mu_j = cal_mu(j)  # Duration of task j

        # Loop through all tasks in the global set Jset to find feasible subsequent tasks
        for j_prime in Jset:
            f_j_prime = cal_f(j_prime)  # Latest feasible start time for task j'
            xi_j_j_prime = cal_xi(j, j_prime)  # Travel time from task j to task j'

            # Check if task j' can start after j considering travel time and its own constraints
            if f_j_prime >= t + mu_j + xi_j_j_prime:
                feasible_tasks.append(j_prime)

        return feasible_tasks

    except KeyError as e:
        raise KeyError(f"Missing data for task calculation: {str(e)}")
    except Exception as e:
        raise Exception(f"An unexpected error occurred: {str(e)}")
    
# Example:
print("The set of tasks that can be performed and finished after task crew pause at Barangaroo if it started at the 100th time period.",cal_taskF('cp_Bar1',100))

The set of tasks that can be performed and finished after task crew pause at Barangaroo if it started at the 100th time period. [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 'CQ1', 'CQ3', 'Bar4', 'Bar5', 'BSY1', 'BSY2', 'BSY3', 'BSY4', 'BSY5', 'BSY6', 'phi_CQ2', 'phi_CQ4', 'phi_CQ5', 'phi_Bar1', 'phi_Bar2', 'phi_Chs1', 'phi_Cab1', 'phi_Pm1', 'phi_WB1', 'phi_RB1', 'phi_CI1', 'phi_PB1', 'cp_CQ1', 'cp_CQ2', 'cp_CQ3', 'cp_CQ4', 'cp_CQ5', 'cp_Bar1', 'cp_Bar2', 'cp_Bar4', 'cp_Bar5', 'cp_BSY1', 'cp_BSY2', 'cp_BSY3', 'cp_BSY4', 'cp_BSY5', 'cp_BSY6', 'CQ2', 'CQ4', 'CQ5', 'Bar1', 'Bar2', 'Chs1', 'Cab1', 'Pm1', 'WB1', 'RB1', 'CI1', 'PB1']


In [35]:
# E(w,t) Function

def cal_E(w, t):
    """
    Calculate the set of pairs (j, t') such that starting task j at time t' results in using a wharf
    at station associated with wharf w at time t.

    Parameters:
    w (str): Wharf identifier.
    t (int): Specific time point.

    Returns:
    set: Set of pairs (j, t') meeting the specified conditions.
    """
    E_wt = []

    for j in Jset:
        C_j = cal_C(j)  # Set of wharves usable for task j
        # print(f'target wharf: {w}, current task {j}, wharf available for task:{C_j}') 
        if w in C_j:
            delta_jw = cal_delta(j, w)  # Set of time units allowed between t' and t for task j and wharf w
            for t_prime in [time for time in Tset if time <= t]:
                if (t - t_prime) in [usage[1] for usage in delta_jw]:
                    E_wt.append((j, t_prime)) # store the task and it's start time

    return E_wt

# Example:
print(f" The wharf PB1 can be occupied at the 10th time period, because of the possible tasks and their start times: {cal_E('PB1', 10)}")
print(f" The wharf Bar1 can be occupied at the 10th time period, because of the possible tasks and their start times: {cal_E('Bar1', 10)}")

 The wharf PB1 can be occupied at the 10th time period, because of the possible tasks and their start times: [('phi_PB1', 10), ('PB1', 10)]
 The wharf Bar1 can be occupied at the 10th time period, because of the possible tasks and their start times: [(5, 5), (5, 6), (5, 7), (5, 8), (7, 3), (7, 4), (7, 5), (7, 6), (8, 7), (8, 8), (8, 9), (8, 10), ('phi_Bar1', 10), ('cp_Bar1', 1), ('cp_Bar1', 2), ('cp_Bar1', 3), ('cp_Bar1', 4), ('cp_Bar1', 5), ('cp_Bar1', 6), ('cp_Bar1', 7), ('cp_Bar1', 8), ('cp_Bar1', 9), ('cp_Bar1', 10), ('Bar1', 10)]


# variables and constraints

In [36]:
import gurobipy as gp
from gurobipy import GRB

In [37]:
# Create model
model = gp.Model("Ferry Timetabling")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-06-26


In [38]:
# Dset: Set of Departures Times
D_l = [cal_D(l) for l in Lset]
Dset = {l: d for l, d in zip(Lset, D_l)}

# variable x[l, d]
x = {}
for l in Lset:
    for d in Dset[l]:
        x[l, d] = model.addVar(vtype=GRB.BINARY, name=f"x_{l}_{d}")

# variable y[v, j, t]
y = {}
for v in Vset:
    for j in Jset:
        for t in Tset:
            y[v, j, t] = model.addVar(vtype=GRB.BINARY, name=f"y_{v}_{j}_{t}")

# variable Q[v, t]
Q = {}
for v in Vset: 
    for t in Tset:
        Q[v, t] = model.addVar(vtype=GRB.CONTINUOUS, lb=0.0, ub=1.0, name=f"Q_{v}_{t}")


# variable z[j, w]
z = {}
for j in Jset:
    C_j = cal_C(j)
    for w in C_j: 
        z[w,j] = model.addVar(vtype=GRB.BINARY, name=f"z_{w}_{j}")


# variable Z[l, w, t]
Z = {}
for l in Lset:
    A_l = cal_Rl(l)[-1]
    C_lS = cal_C_lS(A_l)
    for w in C_lS:
        for t in Tset:
            Z[l, w, t] = model.addVar(vtype=GRB.BINARY, name=f"Z_{l}_{w}_{t}")

# variable Z'[l, w, t]
Z_prime = {}
for l in Lset:
    A_l = cal_Rl(l)[-1]
    C_lS = cal_C_lS(A_l)
    for w in C_lS:
        for t in Tset:
            Z_prime[l, w, t] = model.addVar(vtype=GRB.BINARY, name=f"Z_prime_{l}_{w}_{t}")

In [ ]:
# cal_taskF(j, t), taskF_results
taskF_results = {}
total_combinations = len(Jset) * len(Tset)
processed_combinations = 0

for j in Jset:
    for t in Tset:
        processed_combinations += 1
        taskF_results[(j, t)] = cal_taskF(j, t)
        print(f'Processing ({j}, {t}) - {processed_combinations}/{total_combinations} combinations done.')

with open('pkl_files/taskF_results.pkl', 'wb') as f:
    pickle.dump(taskF_results, f)


# cal_phi(j, t)
phi_results = {}
total_combinations = len(Jset) * len(Tset)
processed_combinations = 0

# Compute and save the results for each combination of j and t
for j in Jset:
    for t in Tset:
        processed_combinations += 1
        phi_results[(j, t)] = cal_phi(j, t)
        print(f'Processing ({j}, {t}) - {processed_combinations}/{total_combinations} combinations done.')

# Save the computed results to a pickle file
with open('pkl_files/phi_results.pkl', 'wb') as f:
    pickle.dump(phi_results, f)

# Precompute cal_mu(j) for each j
mu_results = {j: cal_mu(j) for j in Jset}

# Save to a pickle file
with open('pkl_files/mu_results.pkl', 'wb') as f:
    pickle.dump(mu_results, f)


# cal_xi(j, j_prime)
xi_results = {}
total_combinations = len(Jset) * len(Jset)
processed_combinations = 0

# Compute and save the results for each combination of j and t
for j in Jset:
    for j_prime in Jset:
        processed_combinations += 1
        xi_results[(j, j_prime)] = cal_xi(j, j_prime)
        print(f'Processing ({j}, {j_prime}) - {processed_combinations}/{total_combinations} combinations done.')

# Save the computed results to a pickle file
with open('pkl_files/xi_jj_results.pkl', 'wb') as f:
    pickle.dump(xi_results, f)

# cal_E(w, t)
E_results = {}
total_combinations = len(Wset) * len(Tset)
processed_combinations = 0

# Compute and save the results for each combination of j and t
for w in Wset:
    for t in Tset:
        processed_combinations += 1
        E_results[(w, t)] = cal_E(w, t)
        print(f'Processing ({w}, {t}) - {processed_combinations}/{total_combinations} combinations done.')

# Save the computed results to a pickle file
with open('pkl_files/E_results.pkl', 'wb') as f:
    pickle.dump(E_results, f)

In [39]:
with open('pkl_files/taskF_results.pkl', 'rb') as f:
    taskF_results = pickle.load(f)

with open('pkl_files/mu_results.pkl', 'rb') as f:
    mu_results = pickle.load(f)

with open('pkl_files/xi_jj_results.pkl', 'rb') as f:
    xi_results = pickle.load(f)

with open('pkl_files/phi_results.pkl', 'rb') as f:
    phi_results = pickle.load(f)

with open('pkl_files/E_results.pkl', 'rb') as f:
    E_results = pickle.load(f)

In [40]:
# Constraint 1a

for l in Lset:
    model.addConstr(gp.quicksum(x[l, d] for d in Dset[l]) == 1, name=f"departure_time_constraint_{l}")
    

In [41]:
# Constraint 1b

for sailing in Zset:  
    l = int(sailing.split('_')[0]) # line
    s = int(sailing.split('_')[1]) # nth sailing
    # print(f'line:{l}')
    for d in Dset[l]:  #  
        h_sd = cal_h(s,d,l)
        t = h_sd
        model.addConstr(gp.quicksum(y[v, l, t] for v in Vset) == x[l, d],name=f"assign_vessel_s{s}_d{d}")


In [42]:
# Constraint 1c

for v in Vset:
    for j in Jset:
        H_vj = cal_H(v,j)
        for t in [t for t in Tset if t not in H_vj]:
            y[v, j, t].ub = 0  # Set upper bound of y[v,j,t] to 0

In [43]:
# Constraint 1d

for t in Tset:
    for v in Vset:
        li_v = cal_li(v)
        for j in [l for l in Lset if l not in li_v]:
            y[v, j, t].ub = 0  # Set upper bound of y[v,j,t] to 0

In [44]:
# Constraint 1e
for v in Vset:   
    for j in Jset:
        xi0_v_j = cal_xi0(v,j)
        t = xi0_v_j
        if t in Tset:
            model.addConstr(gp.quicksum(y[v, j, t] for j in Jset) == 1,name=f"assign_task_j{j}_t{t}")

In [45]:
# Constraint 1f 

for v in Vset:
    for t in Tset:
        model.addConstr(gp.quicksum(y[v, j, t_prime] for j in Jset for t_prime in cal_phi(j, t)) <= 1,name=f"task_overlap_v{v}_t{t}")

In [46]:
# Constraint 1g

for l in Lset:
    R_l = cal_Rl(l)
    A_l = R_l[-1] # last station
    for S in [station for station in R_l if station != A_l]:
        C_lS = cal_C_lS(S)
        model.addConstr(gp.quicksum(z[w, l] for w in C_lS) == 1,name=f"select_one_wharf_{l}_station_{S}")

In [47]:
# Constraint 1h

for l in Lset:
    F_l = cal_F(l)
    for t in Tset:
        if t > F_l:
            A_l = cal_Rl(l)[-1] # last station
            C_lS = cal_C_lS(A_l) # available wharves at last station
            model.addConstr(gp.quicksum(Z[l, w, t] for w in C_lS) == gp.quicksum(y[v, l, t - F_l] for v in Vset),name=f"last_wharf_use_{l}_t{t}")

In [48]:
# Constraint 1i

for l in Lset:
    A_l = cal_Rl(l)[-1]
    C_lS = cal_C_lS(A_l)
    for w in C_lS:
        muF_l = cal_muF(l)
        for t in Tset:
            if t > muF_l-1:
                model.addConstr(Z_prime[l, w, t] == gp.quicksum(Z[l, w, t - k] for k in range(muF_l)), name=f"wharf_occupation_{l}_{w}_t{t}")
                


In [ ]:
# Constraint 1j
for v in Vset:
    for w in Bplus:
        for t in Tset: 
            if t > 1:
                phi_w = f'phi_{w}'
                # j = w
                model.addConstr(y[v, w, t] <= y[v, w, t - 1] + y[v, phi_w, t - 1], name=f"full_period_charging_1_v{v}_w{w}_t{t}")

            

In [50]:
# Constraint 1k
for v in Vset:
    for w in Bplus:
        for t in Tset: 
            if t <= Tset[-1]-1:
                phi_w = f'phi_{w}'
                # j = w
                model.addConstr(y[v, w, t] <= y[v, w, t + 1] + y[v, phi_w, t + 1], name=f"full_period_charging_2_v{v}_w{w}_t{t}")

In [ ]:
# Constraint 2 

for v in Vset:
    task_counter = 0
    for j in Jset:
        for t in Tset:
            follow_tasks = taskF_results[(j, t)]
            if follow_tasks != []:
                model.addConstr(gp.quicksum(y[v, j_prime, t + mu_results[j] + xi_results[(j, j_prime)]] for j_prime in follow_tasks) >= y[v, j, t], name=f"follow_task_v{v}_j{j}_t{t}")


In [ ]:

# using pickle version
for v in Vset: 
    for j in Jset:
        for t in Tset:
            for j_prime in taskF_results[(j, t)]:
                for t_prime in range(t + mu_results[j], t + mu_results[j] + xi_results[(j, j_prime)]):
                    if t_prime in Tset:
                        print(f"Current vessel: {v}; current task: {j}; current time {t}; current j'{j_prime}; current t' {t_prime}")
                        model.addConstr(y[v, j, t] + y[v, j_prime, t_prime] <= 1, name=f"no_overlap_v{v}_j{j}_t{t}_j_prime{j_prime}_t_prime{t_prime}")

In [ ]:
rv = {}
for v in Vset:
    rv_value = vessel_df[vessel_df['Vessel code'] == v]['rv'].iloc[0]
    rv[v] = rv_value
print(rv)

{'FF1': 0.041818182, 'FF2': 0.041818182, 'FF3': 0.041818182, 'FF4': 0.041818182, 'FF5': 0.041818182, 'FF6': 0.041818182, 'FF7': 0.041818182, 'FF8': 0.041818182, 'EC1': 0.041818182, 'EC2': 0.106363636, 'EC3': 0.106363636, 'EC4': 0.106363636, 'EC5': 0.106363636, 'RR1': 0.0, 'RR2': 0.0, 'RR3': 0.0, 'RR4': 0.0, 'RR5': 0.0, 'RR6': 0.0, 'RR7': 0.0, 'RR8': 0.0, 'RR9': 0.0, 'RR10': 0.0, 'RR11': 0.0, 'RR12': 0.0, '24M': 0.0, '10M ': 0.041818182}


In [ ]:
# Constraint 4

for w in Wset:
    for t in Tset:
        # Sum over y and z
        sum_yz = gp.quicksum(y[v, j, t_prime] * z[w, j] for v in Vset for (j, t_prime) in E_results[(w, t)])
        # Sum over Z_prime with key check
        sum_Z_prime = gp.quicksum(Z_prime[l, w, t] for l in Lset if (l, w, t) in Z_prime) 
        
        model.addConstr(sum_yz + sum_Z_prime <= cal_Cw(w), name=f"capacity_constraint_w{w}_t{t}")

In [ ]:
# Constraint 5a, 5b

for v in Vset:
    for t in Tset:
        model.addConstr(Q[v, t] >= 0, name=f"battery_non_negative_v{v}_t{t}") 

for v in Vset:
    for t in Tset:
        model.addConstr(Q[v, t] <= 1, name=f"battery_max_capacity_v{v}_t{t}") 


In [ ]:
# Constraint 5c
# This constraint requires a very long time to run. So, I manually killed the terminal here.

for v in Vset:
    for t in Tset:
        rv = vessel_df[vessel_df['Vessel code'] == v]['rv'].iloc()[0]
        if t == 1:
            Qv0 = vessel_df[vessel_df['Vessel code'] == v]['Qv0'].iloc()[0]
            model.addConstr( Qv0 
                        + gp.quicksum(cal_q(v, j, t - t_prime) * y[v, j, t_prime] for j in Jset for t_prime in cal_phi(j, t)) 
                        - rv * (1 - gp.quicksum(y[v, j, t_prime] for j in Jset for t_prime in cal_phi(j, t))) 
                        >= Q[v, t], 
                        name=f"battery_update_v{v}_t{t}")
             
        else:
            model.addConstr(Q[v, t - 1] 
                            + gp.quicksum(cal_q(v, j, t - t_prime) * y[v, j, t_prime] for j in Jset for t_prime in cal_phi(j, t)) 
                            - rv * (1 - gp.quicksum(y[v, j, t_prime] for j in Jset for t_prime in cal_phi(j, t))) 
                            >= Q[v, t], 
                            name=f"battery_update_v{v}_t{t}")



In [ ]:
# Constraint 6a
for v in Vset:
    model.addConstr(gp.quicksum(y[v, j, t] for j in Bc for t in Tset) >= nc, name=f"min_crew_pauses_v{v}")

In [58]:

# Constraint 6b
for v in Vset:
    for t in Tset:
        if t < (Tset[-1] - (Tc // period_length + 1)):
            for t_prime in range(1, Tc // period_length + 1):
                model.addConstr(gp.quicksum(y[v, j, t + t_prime] for j in Bc) >= 1, name=f"max_distance_pauses_v{v}_t{t}_t{t}_t_prime{t_prime}") #12JUl


# Objective functions

In [ ]:
# Variable psi[v]
psi = {}
for v in Vset:
    psi[v] = model.addVar(vtype=GRB.BINARY, name=f"psi_{v}")


In [ ]:
# Objective Function 7
model.setObjective(gp.quicksum(psi[v] for v in Vset), GRB.MINIMIZE)


In [ ]:
# Objective Function 8
M = Tset[-1]
for v in Vset:
    model.addConstr(psi[v] >= (1 / M) * gp.quicksum(y[v, l, t] for l in Lset for t in Tset), name=f"utilize_vessel_{v}")

In [ ]:
# Objective Function 9: Minimizing Rebalancing Time

# This constraint requires a very long time to run. So, I manually killed the terminal here.

rebalancing_time = gp.quicksum(
    1 - gp.quicksum(y[v, j, t_prime] for j in Jset for t_prime in cal_phi(j, t))
    for v in Vset for t in Tset
)
model.setObjective(rebalancing_time, GRB.MINIMIZE)


# # This one is equivalant to the function 9
# # Objective Function 10: Maximizing Time Not Being Rebalanced
# not_rebalancing_time = gp.quicksum(
#     gp.quicksum(y[v, j, t_prime] for j in Jset for t_prime in cal_phi(j, t))
#     for v in Vset for t in Tset
# )
# model.setObjective(not_rebalancing_time, GRB.MAXIMIZE)

In [ ]:
# Optimize the model
model.optimize()